<table style="border: none; border-collapse: collapse;" border="0" cellspacing="0" cellpadding="0" width="100%" align="center">
<tr style="border: none;">
<td  style="border: none;" width="20%">Brought to you by <h3>the dynamic trio</h3></td>
<td style="border: none;"><a href="https://github.com/fredokun/cl-jupyter"><img src="http://jupyter.org/assets/nav_logo.svg" alt="Project Jupyter" height="128"></a></td>
<td style="border: none;"><a href="http://clhs.lisp.se/"><img src="http://www.lisperati.com/lisplogo_fancy_128.png" alt="Public Domain Lisp Logo Set By Conrad Barski, M.D." height="128"></a></td><td style="border: none;"><a href="https://www.hdfgroup.org/"><img src="https://www.hdfgroup.org/images/hdf_logo.jpg" alt="The HDF Group" height="128"></a></td></tr>
</table>

In [ ]:
(require 'asdf)

In [ ]:
(require 'hdf5-cffi)

In [ ]:
(use-package "HDF5")

In [ ]:
(require 'alcove)

In [ ]:
(use-package "ALCOVE")

# An HDF5 file with a user block

The [HDF5 file format specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html) informs us that the superblock might not be at the beginning of an HDF5 file, but elsewhere in the file.

>The superblock may begin at certain predefined offsets within the HDF5 file, allowing a block of unspecified content for users to place additional information at the beginning (and end) of the HDF5 file without limiting the HDF5 Library’s ability to manage the objects within the file itself. This feature was designed to accommodate wrapping an HDF5 file in another file format or adding descriptive information to an HDF5 file without requiring the modification of the actual file’s information. The superblock is located by searching for the HDF5 format signature at byte offset 0, byte offset 512, and at successive locations in the file, each a multiple of two of the previous location; in other words, at these byte offsets: 0, 512, 1024, 2048, and so on.

Let's create an HDF5 file with a userblock of 512 kilobytes (= 524,288 bytes)! All we need is an appropriate [file creation property list](https://www.hdfgroup.org/HDF5/doc/RM/RM_H5P.html#Property-SetUserblock).

In [ ]:
(defparameter *file-name* "userblock.h5")

In [ ]:
(let ((fcpl (h5pcreate +H5P-FILE-CREATE+)))
     (h5pset-userblock fcpl (cffi:convert-to-foreign 524288 'hsize-t))
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ fcpl +H5P-DEFAULT+))  ; create and close the file
     (h5pclose fcpl))

Let's find the file format signature and read the superblock!

In [ ]:
(defparameter *input-stream*
    (open *file-name* :direction :input :element-type '(unsigned-byte 8)))

In [ ]:
(documentation 'find-format-signature 'function)

The function ``FIND-FORMAT-SIGNATURE`` is implemented as a simple recursion:

```lisp
(defun find-format-signature (input-stream
                              &optional (max-offset
                                         (file-length input-stream)))
  (assert (file-position input-stream 0))
  ;; define a helper function to recursively look for the signature
  (labels ((find-it (in-stream offset)
             (let ((buffer (make-array 8 :element-type 'unsigned-byte))
                   (offset (file-position input-stream)))
               (when (and offset
                          (< (+ offset 8) max-offset)
                          (read-sequence buffer input-stream))
                 (if (every #'= +hdf5-signature+ buffer)
                     offset
                     (when (and (setq offset (if (< 0 offset)
                                                 (* 2 offset)
                                                 512))
                                (< offset max-offset)
                                (file-position input-stream offset))
                       (find-it input-stream max-offset)))))))
    (find-it input-stream max-offset)))
```

In [ ]:
(find-format-signature *input-stream*)

In [ ]:
(read-superblock *input-stream*)

(Remember that 18,446,744,073,709,551,615 is the *undefined address* $2^{64}-1$ ?)

Notice the ``BASE-ADDRESS`` is 524,288 (instead of 0).

>Unless otherwise noted, all other file addresses are relative to this base address.

That means the root group's object header is not at offset 96, but at 524,384 (= 524,288 + 96).

In [ ]:
(close *input-stream*)

In the next installment, we'll read that object header.